In [ ]:
import pandas as pd
import math
from happi import Client
from bmad_components.containers import Monitor, Instrument, Ecollimator, Rcollimator, Mirror, Marker

In [ ]:
xl = pd.ExcelFile("TXIComponents_2020-08-24.xlsx")

In [ ]:
print(xl.sheet_names)

In [ ]:
components = xl.parse('ComponentList')
components.keys()

In [ ]:
# Drop all rows without X, Y, Z
components = components[components['X-LCLS [m]\nY (SE)'].notnull()]
components

In [ ]:
# build devices
devices = []
count = 0
for row in components.iterrows():
   # print(row[1]["Component/System"])
    comp_name = row[1]["Component/System"]
    branch_line = row[1]["Branch Line"]
    location = row[1]["Location"]
    X_OFFSET = row[1]["X-LCLS [m]\nY (SE)"]
    Y_OFFSET = row[1]["Y-LCLS [m]\nZ (SE)"]
    Z_OFFSET = row[1]["Z-LCLS [m]\nX (SE)"]
    label = row[1]["Label"]
    
    description = row[1]["Description"]
    
    #filter nan
    if isinstance(description, float):
        if math.isnan(description):
            description = None
        
    pitch = row[1]["Pitch (rad)"]
    
    #filter nan
    if isinstance(label, float):
        if math.isnan(label):
            label = "placeholder_label" + str(count)
            count += 1
            
    if not isinstance(comp_name, float):
        
        happi_class = None
        if "Monitor" in comp_name:
            happi_class = Monitor

        elif "Valve" in comp_name:
            #Using instrument for now...
            happi_class = Instrument

        elif "Collimator" in comp_name:
            # Using ecollimator for now...
            happi_class = Ecollimator

        elif "Stopper" in comp_name:
            # using instrument for now
            happi_class = Instrument

        elif "Mirror" in comp_name: 
            happi_class = Mirror

        elif "Delay-Line" in comp_name:
            happi_class = Instrument

        elif "Mirror System" in comp_name:
            # using mirror for now
            happi_class = Mirror

        elif "Terminator" in comp_name:
            happi_class = Instrument

        elif "Focal Position" in comp_name:
            happi_class = Marker

        elif "wall surface" in comp_name:
            pass

        elif "Wall" in comp_name:
            pass

        elif "Timing" in comp_name:
            happi_class = Instrument

        elif "Wavefront Sensor" in comp_name:
            happi_class = Monitor
            
        if happi_class is not None:

            if not description:
                device = happi_class(
                            X_OFFSET=X_OFFSET, 
                            Y_OFFSET=Y_OFFSET, 
                            Z_OFFSET=Z_OFFSET, 
                            branch_line=branch_line,
                            name=label,
                            L=0,
                        )
            else:
                device = happi_class(
                            X_OFFSET=X_OFFSET, 
                            Y_OFFSET=Y_OFFSET, 
                            Z_OFFSET=Z_OFFSET, 
                            branch_line=branch_line,
                            name=label,
                            description=description,
                            L=0,
                        )

            devices.append(device)




In [ ]:
client = Client(path="txi_db.json")

for device in devices:
    try:
        client.add_device(device)
    except Exception as e:
        print(e)